In [1]:
import pandas as pd
from functools import reduce
from datetime import timedelta
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Read datasets
olist_customers_dataset = pd.read_csv("D:\\IML\\projet4\\Data\\olist_customers_dataset.csv")
olist_order_items_dataset = pd.read_csv("D:\\IML\\projet4\\Data\\olist_order_items_dataset.csv")
olist_order_payments_dataset = pd.read_csv("D:\\IML\\projet4\\Data\\olist_order_payments_dataset.csv")
olist_order_reviews_dataset = pd.read_csv("D:\\IML\\projet4\\Data\\olist_order_reviews_dataset.csv")
olist_orders_dataset = pd.read_csv("D:\\IML\\projet4\\Data\\olist_orders_dataset.csv")
olist_products_dataset = pd.read_csv("D:\\IML\\projet4\\Data\\olist_products_dataset.csv")
olist_sellers_dataset = pd.read_csv("D:\\IML\\projet4\\Data\\olist_sellers_dataset.csv")

Nous allons parcourir chaque base de données afin de voire les relations entres elles.

In [3]:
# Customer dataset
print('La table des clients comporte {} commandes et {} de variables' \
      .format(olist_customers_dataset.shape[0], 
              olist_customers_dataset.shape[1]))

print('La table des clients comporte au total {} clients' \
      .format(olist_customers_dataset.customer_unique_id.unique().shape[0]))

La table des clients comporte 99441 commandes et 5 de variables
La table des clients comporte au total 96096 clients


La base de données Clients nous montre que certains clients ont effectués plus d'une commande. En effet sur les 96096 clients de notre base de données, nous avons enregistré 99441 commandes. Nous allons créer une variable qui va capturer le nombre commandes effectué par un client.

In [4]:
# Orders dataset
print('La table des commandes comporte {} commandes et {} de variables' \
      .format(olist_orders_dataset.shape[0], 
              olist_orders_dataset.shape[1]))

La table des commandes comporte 99441 commandes et 8 de variables


In [5]:
# Order_items dataset
print('La table des commandes des produits comporte {} de produits commandés et {} de variables' \
      .format(olist_order_items_dataset.shape[0], 
              olist_order_items_dataset.shape[1]))

print('La table des commandes des produits comporte {} commandes' \
      .format(olist_order_items_dataset.order_id.unique().shape[0]))

print('La table des commandes des produits comporte {} types de produits commandés  différents' \
      .format(olist_order_items_dataset.product_id.unique().shape[0]))

print('La table des commandes des produits comporte {} types de vendeurs différents' \
      .format(olist_order_items_dataset.seller_id.unique().shape[0]))

La table des commandes des produits comporte 112650 de produits commandés et 7 de variables
La table des commandes des produits comporte 98666 commandes
La table des commandes des produits comporte 32951 types de produits commandés  différents
La table des commandes des produits comporte 3095 types de vendeurs différents


Nous avons plus de produits commandés (112650) que de commandes effectués(98666). Nous allons créer une variables qui va captuer le nombre de produits dans chaque commande.

En outre, nous constatons qu'au lieu de 99441 commandes nous avons seulement 98666 commandes. Donc nous avons des commandes qui ne figurent pas dans la base de données commandes des produits. Cela va entrainer que nous n'aurons pas entre autre des informations sur les prix des produits contituant chacune de ces commandes. Nous allons supprimer ces commandes de notre analyse.

Nous allons aussi créer des variables pour capturer le prix total et les frais de port associés à un type de produit donné dans une commande.

In [6]:
# Order payment dataset
print('La table des payements des commandes comporte {} nombres de payement et {} de variables' \
      .format(olist_order_payments_dataset.shape[0], 
              olist_order_payments_dataset.shape[1]))

print('La table des payements des commandes comporte {} commandes' \
      .format(olist_order_payments_dataset.order_id.unique().shape[0]))

La table des payements des commandes comporte 103886 nombres de payement et 5 de variables
La table des payements des commandes comporte 99440 commandes


Nous avons plus d'opérations de payement (103056) que de commandes effectués(98665). Nous allons créer une variables qui va captuer le nombre d'opérations de payement pour chaque commande.

En outre, nous avons dans cette base de données 98665 commandes au lieu de 98666 commandes espérées. Donc nous avons une commande dont nous n'avons pas d'informations sur les méthodes de payement. Nous allons donc supprimer cette commande de notre étude.

In [7]:
# Reviews dataset
print('La table des reviews comporte {} reviews et {} de variables' \
      .format(olist_order_reviews_dataset.shape[0], 
              olist_order_reviews_dataset.shape[1]))

print('La table des des reviews comporte {} commandes' \
      .format(olist_order_reviews_dataset.order_id.unique().shape[0]))

print('La table des reviews comporte {} uniques reviews' \
      .format(olist_order_reviews_dataset.review_id.unique().shape[0]))

La table des reviews comporte 100000 reviews et 7 de variables
La table des des reviews comporte 99441 commandes
La table des reviews comporte 99173 uniques reviews


In [8]:
# Reviews dataset
''' Est ce qu'un review peut s'addresser à plusieurs commandes?'''
ReviewToOrders = olist_order_reviews_dataset.groupby('review_id').size().sort_values()
ReviewToOrders3 = ReviewToOrders[ReviewToOrders.values == 3].index.tolist()
olist_order_reviews_dataset[olist_order_reviews_dataset.review_id == ReviewToOrders3[0]]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
24899,39b4603793c1c7f5f36d809b4a218664,2613fb342bec59126f9c5180a5bc95ec,5,NaN,Otimo,2017-09-01 00:00:00,2017-09-02 12:13:03
28491,39b4603793c1c7f5f36d809b4a218664,2d4bc14df7f5eaf36d2ef6d9a5b7c0d8,5,NaN,Otimo,2017-09-01 00:00:00,2017-09-02 12:13:03
42826,39b4603793c1c7f5f36d809b4a218664,02e723e8edb4a123d414f56cc9c4665e,5,NaN,Otimo,2017-09-01 00:00:00,2017-09-02 12:13:03


Nous constatons qu'un client peut faire un même review pour plusieurs commandes.

In [9]:
''' Est ce qu'une commande peut recevoir à plusieurs reviews?'''
OrderGetReviews = olist_order_reviews_dataset.groupby('order_id').size().sort_values()
OrderGetReviews3 = OrderGetReviews[OrderGetReviews.values == 3].index.tolist()
olist_order_reviews_dataset[olist_order_reviews_dataset.order_id == OrderGetReviews3[0]]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
2970,c444278834184f72b1484dfe47de7f97,df56136b8031ecd28e200bb18e6ddb2e,5,NaN,NaN,2017-02-08 00:00:00,2017-02-14 13:58:48
14084,72a1098d5b410ae50fbc0509d26daeb9,df56136b8031ecd28e200bb18e6ddb2e,5,NaN,NaN,2017-02-07 00:00:00,2017-02-10 10:46:09
63225,44f3e54834d23c5570c1d010824d4d59,df56136b8031ecd28e200bb18e6ddb2e,5,NaN,NaN,2017-02-09 00:00:00,2017-02-09 09:07:28


Nous constatons qu'un client peut faire plusieurs reviews sur une commande

Au vu de ce qui précède, nous allons créer desux variables pour capturer d'une part le nombre de reviews qu'une commande a reçu et d'autre part, le nombre de commandes auxquelles s'adresse un review donné.

In [10]:
# Products dataset
print('La table des produits comporte {} produits et {} de variables' \
      .format(olist_products_dataset.shape[0], 
              olist_products_dataset.shape[1]))

La table des produits comporte 32951 produits et 9 de variables


In [11]:
# Sellers dataset
print('La table des vendeurs comporte {} vendeurs et {} de variables' \
      .format(olist_sellers_dataset.shape[0], 
              olist_sellers_dataset.shape[1]))

La table des vendeurs comporte 3095 vendeurs et 4 de variables
